### Proyecto de Pronóstico de Energía.  
Region Noroeste de Mexico.

### Initial analysis for Electricity load demand.
North- West of Mexico





La presente libreta incluye los pasos de pre proceso de nuestro análisis, considerando las siguientes fuentes:

01. de Cenace, DemandaGCRN020221031.csv
02. de Meteomatics, 'real_met.csv' which cointains Data From Sonora and Sinaloa

The present document includes the preprocessing of raw sources for our analysis.

01. From Cenace, DemandaGCRN020221031.csv
02. From Meteomatics, 'real_met.csv' which cointains Data From Sonora and Sinaloa



In [1]:
#import libraries
import pandas as pd
import os
import urllib
import requests    
import wget
import zipfile
from datetime import datetime
import shutil

In [2]:
#set working directory  

print(os.getcwd());

/Users/yhmve/nrgy_feng/2023_wip


### Archivo 1.
El archivo 1, contiene los datos de demanda de Energía real, por cada hora del día, desde el 01/01/2007 al 10/31/2022 00:00.

First file reviewed contains actual Demand Energy from  01/01/2007 to 10/31/2022 00:00.

In [3]:
#exploring available files 01
df_demanda = pd.read_csv('DemandaGCRNO20221031.csv', delimiter=",")
df_demanda.tail(2)

,FECHA,H1,H2,H3,H4,H5,H6,H7,H8,H9,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
5781,30/10/2022,2629.53,2535.63,2450.04,2359.91,2294.59,2264.86,2231.41,2168.14,2165.34,...,2618.61,2656.02,2696.42,2690.23,2655.08,2750.37,2776.64,2809.72,2773.22,2680.07
5782,31/10/2022,2566.49,2467.76,2383.80,2317.80,2257.29,2241.29,2257.35,2256.47,2320.47,...,2914.07,2990.75,3038.08,3014.08,2934.74,2926.85,2894.38,2877.56,2843.08,2772.44


In [4]:
#check for null values
df_demanda.isnull().sum() # check for null values, there is none
df_demanda.describe() #review 

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,...,H15,H16,H17,H18,H19,H20,H21,H22,H23,H24
count,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,...,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000,5783.000000
mean,2587.026291,2482.607166,2395.337398,2321.298703,2259.878649,2218.953688,2176.724837,2140.685528,2195.350645,2296.725708,...,2676.137047,2733.599877,2742.815923,2699.388506,2632.013244,2627.109064,2700.725637,2763.938153,2759.916894,2689.676913
std,901.941794,874.350368,839.920596,803.437776,764.184118,723.829645,658.376044,597.775994,614.682505,645.262486,...,869.747305,915.021938,926.094018,902.124466,837.778588,759.928137,777.631661,850.288766,899.233689,913.746691
min,1105.000000,1059.000000,1032.000000,1009.000000,1003.000000,1007.000000,1037.000000,1023.000000,959.000000,966.000000,...,1003.000000,1001.000000,1002.000000,1025.000000,1112.000000,1302.000000,1333.000000,1327.000000,1266.000000,1190.000000
25%,1830.500000,1750.155000,1693.370000,1654.040000,1635.150000,1631.400000,1654.175000,1676.670000,1724.170000,1802.220000,...,1998.000000,2016.000000,2016.000000,1993.005000,1986.000000,2050.050000,2096.000000,2075.465000,2012.500000,1922.055000
50%,2408.810000,2305.000000,2221.000000,2145.290000,2093.000000,2073.150000,2052.000000,2040.000000,2097.000000,2194.510000,...,2480.000000,2527.000000,2537.000000,2499.000000,2437.510000,2438.000000,2501.500000,2559.000000,2558.000000,2499.000000
75%,3290.500000,3163.890000,3042.585000,2934.000000,2836.000000,2754.000000,2640.500000,2532.465000,2594.415000,2717.675000,...,3258.500000,3357.220000,3375.795000,3313.500000,3192.910000,3122.310000,3226.610000,3386.500000,3452.000000,3401.905000
max,4950.140000,4765.340000,4616.920000,4479.680000,4359.280000,4265.730000,4132.320000,3994.670000,4049.500000,4195.830000,...,5268.000000,5390.000000,5402.720000,5336.530000,5164.980000,4975.850000,5034.310000,5211.710000,5226.670000,5124.170000


In [5]:
# Transpose and convert hour columns to rows
deman_td = df_demanda.melt(
    
    id_vars= ['FECHA'],
    value_vars= [f'H{i}' for i in range(1,24)],
    var_name="Hour",
    value_name="Demanda").replace(
    {f'H{i}': i for i in range(1,24)}
)

In [6]:
#Formatting column names
deman_td.rename(columns={"FECHA":"Date_dem","Demanda":"Energy_demand"}, inplace = True)
deman_td.head(2)

,Date_dem,Hour,Energy_demand
0,01/01/2007,1,1297.0
1,02/01/2007,1,1177.0


In [7]:
# Adjust Date to correct datatype
deman_td['Date_dem'] =  pd.to_datetime(deman_td['Date_dem'], format='%d/%m/%Y')

In [8]:
#Adjust datetime column
deman_td['Date_time'] = deman_td.Date_dem + pd.to_timedelta(deman_td.Hour, unit='h')
deman_td.info()

<bound method DataFrame.info of          Date_dem  Hour  Energy_demand           Date_time
0      2007-01-01     1        1297.00 2007-01-01 01:00:00
1      2007-01-02     1        1177.00 2007-01-02 01:00:00
2      2007-01-03     1        1376.00 2007-01-03 01:00:00
3      2007-01-04     1        1390.00 2007-01-04 01:00:00
4      2007-01-05     1        1394.00 2007-01-05 01:00:00
...           ...   ...            ...                 ...
133004 2022-10-27    23        3089.08 2022-10-27 23:00:00
133005 2022-10-28    23        3055.46 2022-10-28 23:00:00
133006 2022-10-29    23        2825.81 2022-10-29 23:00:00
133007 2022-10-30    23        2773.22 2022-10-30 23:00:00
133008 2022-10-31    23        2843.08 2022-10-31 23:00:00

[133009 rows x 4 columns]>

In [9]:
# Creating Day, Hour and Month columns
deman_td.index = deman_td.Date_dem + pd.to_timedelta(deman_td.Hour, unit='h')
deman_td.sort_index(inplace=True)
deman_td.drop(columns=['Hour'], inplace=True)
deman_td = deman_td.asfreq('h', method='pad')
deman_td['Date_time'] = deman_td.index
deman_td["Hour"] = deman_td.index.hour
deman_td["Week_day"] = deman_td.index.weekday
deman_td["Week"] = deman_td.index.week
deman_td["Month"] = deman_td.index.month  
deman_td["Year"] = deman_td.index.year

/var/folders/nf/4xpfszv92kn0gqccmstgzn840000gn/T/ipykernel_35096/2748183332.py:9: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  deman_td["Week"] = deman_td.index.week


In [11]:
#deman_td.head(30)
deman_td['Date_dem']= deman_td['Date_dem'].shift(-1)

In [12]:
deman_td.tail(2)

,Date_dem,Energy_demand,Date_time,Hour,Week_day,Week,Month,Year
2022-10-31 22:00:00,2022-10-31,2877.56,2022-10-31 22:00:00,22,0,44,10,2022
2022-10-31 23:00:00,NaT,2843.08,2022-10-31 23:00:00,23,0,44,10,2022


In [13]:
deman_td.loc['2022-10-31 23:00:00', 'Date_dem'] = '2022-10-31'

In [14]:
# function to add season column
def find_season(month):
    dseason = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
    return dseason.get(month)

season_list = []
#hemisphere = 'Southern'
for month in deman_td['Month']:
    season = find_season(month)
    season_list.append(season)

deman_td['Season'] = season_list
deman_td.head()

,Date_dem,Energy_demand,Date_time,Hour,Week_day,Week,Month,Year,Season
2007-01-01 01:00:00,2007-01-01,1297.0,2007-01-01 01:00:00,1,0,1,1,2007,Winter
2007-01-01 02:00:00,2007-01-01,1255.0,2007-01-01 02:00:00,2,0,1,1,2007,Winter
2007-01-01 03:00:00,2007-01-01,1222.0,2007-01-01 03:00:00,3,0,1,1,2007,Winter
2007-01-01 04:00:00,2007-01-01,1168.0,2007-01-01 04:00:00,4,0,1,1,2007,Winter
2007-01-01 05:00:00,2007-01-01,1128.0,2007-01-01 05:00:00,5,0,1,1,2007,Winter


Import Meteomatics files:


In [15]:
print(os.getcwd())

/Users/yhmve/nrgy_feng/2023_wip


In [17]:
os.chdir('./Meteomatics') #move to correct directory
print(os.getcwd())

/Users/yhmve/nrgy_feng/2023_wip/Meteomatics


In [18]:
#importing city files to each df
df_cab = pd.read_csv('Cab_20221031.csv')
df_cul = pd.read_csv('Cul_20221031.csv')
df_guas = pd.read_csv('Guas_20221031.csv')
df_guay = pd.read_csv('Guay_20221031.csv')
df_herm = pd.read_csv('Herm_20221031.csv')
df_maz = pd.read_csv('Maz_20221031.csv')
df_moch = pd.read_csv('Moch_20221031.csv')
df_nav = pd.read_csv('Nav_20221031.csv')
df_nog = pd.read_csv('Nog_20221031.csv')
df_obr = pd.read_csv('Obr_20221031.csv')

In [19]:
df_cab.head(2)

,Gerencia,Zona_Carga,Estacion,API,Fecha,Hora,Temperatura,Temperatura_Aparente,Precipitacion,Humedad,Velocidad_Viento,Radiacion_Solar,Nubosidad
0,GCRNO,Caborca,Caborca PPA,Meteomatics,01/01/2010,0,12.7,11.3,0.0,30.9,2.6,0.0,0.0
1,GCRNO,Caborca,Caborca PPA,Meteomatics,01/01/2010,1,11.6,10.8,0.0,32.8,2.6,0.0,0.0


In [53]:
#Check shape of all df
#create dataframe list
data_frames = [df_cab, df_cul, df_guas, df_guay, df_herm , df_maz, df_moch, df_nav, df_nog , df_obr]

# Loop through the list of data frames
for i, df in enumerate(data_frames):
    print(f"Dataframe {i+1} shape: {df.shape}")


Dataframe 1 shape: (112428, 10)
Dataframe 2 shape: (112428, 10)
Dataframe 3 shape: (112428, 10)
Dataframe 4 shape: (112428, 10)
Dataframe 5 shape: (112405, 10)
Dataframe 6 shape: (112428, 10)
Dataframe 7 shape: (112428, 10)
Dataframe 8 shape: (112428, 10)
Dataframe 9 shape: (112428, 10)
Dataframe 10 shape: (112428, 10)


In the above review we found that DF 5 (HMO) is missing some data vs the other cities

In [20]:
df_cab.columns #verify columns in DF

Index(['Gerencia', 'Zona_Carga', 'Estacion', 'API', 'Fecha', 'Hora',
       'Temperatura', 'Temperatura_Aparente', 'Precipitacion', 'Humedad',
       'Velocidad_Viento', 'Radiacion_Solar', 'Nubosidad'],
      dtype='object')

In [21]:
"""" Function to update columns"""

def drop(name): #drop unused columns
    name.drop(['Gerencia','Estacion', 'API'], axis=1, inplace=True)
    name.head(2)

def change_names(name): #translate column names
    cnames = ['City','Date','Hour','Temperature','Perceived_Temperature','Precipitation',
         'Humidity','Wind_Velocity','Solar_Radiation','Cloudiness']
    name.set_axis(cnames, axis = 1,  inplace = True)    

def upd_names(name,suf): # add sufix to columns
    name.columns = name.columns.map(lambda x : x + suf)

def main_update(name, suf): # main 
    drop(name)
    change_names(name)
    upd_names(name,suf)
    name.columns

In [22]:
#Applying function to all cities
main_update (df_cab,"_Ca")
main_update (df_cul,"_Cul")
main_update (df_guas,"_Guas")
main_update (df_guay,"_Guay")
main_update (df_herm,"_Herm")
main_update (df_maz,"_Maz")
main_update (df_moch,"_Moch")
main_update (df_nav,"_Nav")
main_update (df_nog,"_Nog")
main_update (df_obr,"_Obr")

In [24]:
df_cab.columns # verify columns have imported

Index(['City_Ca', 'Date_Ca', 'Hour_Ca', 'Temperature_Ca',
       'Perceived_Temperature_Ca', 'Precipitation_Ca', 'Humidity_Ca',
       'Wind_Velocity_Ca', 'Solar_Radiation_Ca', 'Cloudiness_Ca'],
      dtype='object')

In [25]:
#Renaming date and hour for merges    
df_cab.rename(columns = {'Date_Ca':'Date', 'Hour_Ca':'Hour'}, inplace = True)
df_herm.rename(columns = {'Date_Herm':'Date', 'Hour_Herm':'Hour'}, inplace = True)
df_cul.rename(columns = {'Date_Cul':'Date', 'Hour_Cul':'Hour'}, inplace = True)
df_guas.rename(columns = {'Date_Guas':'Date', 'Hour_Guas':'Hour'}, inplace = True)
df_guay.rename(columns = {'Date_Guay':'Date', 'Hour_Guay':'Hour'}, inplace = True)
df_maz.rename(columns = {'Date_Maz':'Date', 'Hour_Maz':'Hour'}, inplace = True)
df_moch.rename(columns = {'Date_Moch':'Date', 'Hour_Moch':'Hour'}, inplace = True)
df_nav.rename(columns = {'Date_Nav':'Date', 'Hour_Nav':'Hour'}, inplace = True)
df_nog.rename(columns = {'Date_Nog':'Date', 'Hour_Nog':'Hour'}, inplace = True)
df_obr.rename(columns = {'Date_Obr':'Date', 'Hour_Obr':'Hour'}, inplace = True)

In [26]:
#Merge happens on both columns Date & hour,case Hermosillo + caborca
df_hermcab = df_cab.merge(df_herm, left_on=['Date','Hour'], right_on = ['Date','Hour'], how='left')
 

In [27]:
df_hermcab.isnull().sum() #check nulls according to the different shapes of original DF
#df_mtrl.isnull().sum()
#df_hermcab.shape

City_Ca                        0
Date                           0
Hour                           0
Temperature_Ca                 0
Perceived_Temperature_Ca       0
Precipitation_Ca               0
Humidity_Ca                    0
Wind_Velocity_Ca               0
Solar_Radiation_Ca             0
Cloudiness_Ca                  0
City_Herm                     23
Temperature_Herm              23
Perceived_Temperature_Herm    23
Precipitation_Herm            23
Humidity_Herm                 23
Wind_Velocity_Herm            23
Solar_Radiation_Herm          23
Cloudiness_Herm               23
dtype: int64

In [28]:
new_df = [] #set new df

In [29]:
#Join all dataframes_list but this version duplicates the columns Date and hour
#df_hermcab = df_cab.merge(df_herm, left_on=['Date','Hour'], right_on = ['Date','Hour'], how='left')

pdlist = [df_hermcab,df_cul,df_guas, df_guay, df_maz,df_moch,df_nav,df_nog,df_obr ]
new_df = pd.concat((pdlist),axis = 1, join ='outer')

In [30]:
new_df.tail(24)

,City_Ca,Date,Hour,Temperature_Ca,Perceived_Temperature_Ca,Precipitation_Ca,Humidity_Ca,Wind_Velocity_Ca,Solar_Radiation_Ca,Cloudiness_Ca,...,City_Obr,Date,Hour,Temperature_Obr,Perceived_Temperature_Obr,Precipitation_Obr,Humidity_Obr,Wind_Velocity_Obr,Solar_Radiation_Obr,Cloudiness_Obr
112404,Caborca,30/10/2022,0,19.2,15.9,0.0,34.3,2.3,0.0,0.0,...,Obregon,30/10/2022,0,20.7,20.3,0.0,63.4,2.1,0.0,0.0
112405,Caborca,30/10/2022,1,18.1,15.7,0.0,38.6,1.5,0.0,0.0,...,Obregon,30/10/2022,1,19.4,19.5,0.0,70.0,2.1,0.0,0.0
112406,Caborca,30/10/2022,2,17.6,16.2,0.0,40.8,0.6,0.0,0.0,...,Obregon,30/10/2022,2,16.8,17.3,0.0,75.8,1.4,0.0,0.0
112407,Caborca,30/10/2022,3,15.9,14.5,0.0,45.2,1.4,0.0,0.0,...,Obregon,30/10/2022,3,15.7,16.2,0.0,75.8,1.2,0.0,0.0
112408,Caborca,30/10/2022,4,14.2,13.0,0.0,48.6,2.5,0.0,0.0,...,Obregon,30/10/2022,4,15.1,15.6,0.0,73.6,0.9,0.0,0.0
112409,Caborca,30/10/2022,5,13.5,12.3,0.0,46.5,2.9,0.0,0.0,...,Obregon,30/10/2022,5,14.9,15.2,0.0,71.5,0.9,0.0,0.0
112410,Caborca,30/10/2022,6,12.9,11.8,0.0,43.8,3.0,0.0,0.0,...,Obregon,30/10/2022,6,14.7,14.9,0.0,70.3,1.0,0.0,0.0
112411,Caborca,30/10/2022,7,12.0,11.2,0.0,44.8,3.1,0.0,0.0,...,Obregon,30/10/2022,7,13.9,14.0,0.0,70.6,1.1,0.0,0.0
112412,Caborca,30/10/2022,8,11.3,10.8,0.0,46.5,3.1,0.0,0.0,...,Obregon,30/10/2022,8,11.0,14.0,0.0,75.0,0.5,0.0,0.0
112413,Caborca,30/10/2022,9,12.7,12.6,0.0,41.0,2.8,101.1,0.0,...,Obregon,30/10/2022,9,16.2,17.8,0.0,63.2,0.0,165.4,0.0


In [31]:
#join sets by Date & Hour
dfs = (df_cul.set_index(['Date','Hour'])) #first df and set key columns as indx for new dataframe

In [32]:
dfs = pd.merge(dfs, df_hermcab,  how='left', left_on=['Date','Hour'], right_on=['Date','Hour'] ) #merge by Date Hour
dfs.head(3)

,Date,Hour,City_Cul,Temperature_Cul,Perceived_Temperature_Cul,Precipitation_Cul,Humidity_Cul,Wind_Velocity_Cul,Solar_Radiation_Cul,Cloudiness_Cul,...,Solar_Radiation_Ca,Cloudiness_Ca,City_Herm,Temperature_Herm,Perceived_Temperature_Herm,Precipitation_Herm,Humidity_Herm,Wind_Velocity_Herm,Solar_Radiation_Herm,Cloudiness_Herm
0,01/01/2010,0,Culiacan,16.9,17.1,0.0,73.0,1.7,0.0,0.0,...,0.0,0.0,Hermosillo,13.3,12.1,0.0,46.5,3.0,0.0,0.0
1,01/01/2010,1,Culiacan,16.5,17.0,0.0,76.3,1.3,0.0,0.0,...,0.0,0.0,Hermosillo,12.5,11.6,0.0,48.1,3.0,0.0,0.0
2,01/01/2010,2,Culiacan,13.3,13.7,0.0,82.3,1.2,0.0,0.0,...,0.0,0.0,Hermosillo,11.3,10.8,0.0,50.6,3.0,0.0,0.0


In [33]:
#Merge all city_dataframes from list, based on columns date hour (without duplicating the columns)
from functools import reduce
pdlist2 = [dfs,df_guas, df_guay, df_maz,df_moch,df_nav,df_nog,df_obr ]
df_ = reduce(lambda dfs,df_guas: pd.merge(dfs,df_guas,on=['Date','Hour']), pdlist2)
df_.shape

(112428, 82)

In [34]:
deman_td.head(2) #verify nrg df for next union

,Date_dem,Energy_demand,Date_time,Hour,Week_day,Week,Month,Year,Season
2007-01-01 01:00:00,2007-01-01,1297.0,2007-01-01 01:00:00,1,0,1,1,2007,Winter
2007-01-01 02:00:00,2007-01-01,1255.0,2007-01-01 02:00:00,2,0,1,1,2007,Winter


In [35]:
df_.tail(2) #verify all cities df for next union

,Date,Hour,City_Cul,Temperature_Cul,Perceived_Temperature_Cul,Precipitation_Cul,Humidity_Cul,Wind_Velocity_Cul,Solar_Radiation_Cul,Cloudiness_Cul,...,Solar_Radiation_Nog,Cloudiness_Nog,City_Obr,Temperature_Obr,Perceived_Temperature_Obr,Precipitation_Obr,Humidity_Obr,Wind_Velocity_Obr,Solar_Radiation_Obr,Cloudiness_Obr
112426,30/10/2022,22,Culiacan,23.0,21.9,0.0,66.3,0.0,0.0,0.0,...,0.0,0.0,Obregon,19.8,18.2,0.0,52.0,2.1,0.0,0.0
112427,30/10/2022,23,Culiacan,23.0,21.4,0.0,68.2,0.0,0.0,0.0,...,0.0,0.0,Obregon,19.3,18.4,0.0,58.7,1.9,0.0,0.0


In [36]:
df_.head(3)

,Date,Hour,City_Cul,Temperature_Cul,Perceived_Temperature_Cul,Precipitation_Cul,Humidity_Cul,Wind_Velocity_Cul,Solar_Radiation_Cul,Cloudiness_Cul,...,Solar_Radiation_Nog,Cloudiness_Nog,City_Obr,Temperature_Obr,Perceived_Temperature_Obr,Precipitation_Obr,Humidity_Obr,Wind_Velocity_Obr,Solar_Radiation_Obr,Cloudiness_Obr
0,01/01/2010,0,Culiacan,16.9,17.1,0.0,73.0,1.7,0.0,0.0,...,0.0,0.0,Obregon,14.7,13.7,0.0,58.9,2.9,0.0,0.0
1,01/01/2010,1,Culiacan,16.5,17.0,0.0,76.3,1.3,0.0,0.0,...,0.0,0.0,Obregon,14.3,13.6,0.0,61.5,2.6,0.0,0.0
2,01/01/2010,2,Culiacan,13.3,13.7,0.0,82.3,1.2,0.0,0.0,...,0.0,0.0,Obregon,12.5,12.1,0.0,64.8,2.5,0.0,0.0


In [38]:
#creating copy to modify datatype and merge
df_n = df_.copy(deep = True)


In [39]:

df_n['Date'] = pd.to_datetime(df_n['Date'], format='%d/%m/%Y')

In [40]:
#final adjustment to energy df 
#adjust columns on the same format for merge
deman_td= deman_td.reset_index (drop=True)
deman_td.rename(columns = {'Date_dem':'Date'}, inplace = True)
deman_td.head(2)

,Date,Energy_demand,Date_time,Hour,Week_day,Week,Month,Year,Season
0,2007-01-01,1297.0,2007-01-01 01:00:00,1,0,1,1,2007,Winter
1,2007-01-01,1255.0,2007-01-01 02:00:00,2,0,1,1,2007,Winter


In [41]:
deman_td.shape

(138791, 9)

In [42]:
#review of data found on first day of available data
deman_td.iloc[[24]]

,Date,Energy_demand,Date_time,Hour,Week_day,Week,Month,Year,Season
24,2007-01-02,1177.0,2007-01-02 01:00:00,1,1,1,1,2007,Winter


In [43]:
#join cities dataframe with nrgy requirement dataset
df_f = pd.merge(deman_td, df_n,  how='left', left_on=['Date','Hour'],right_on=['Date','Hour']) #merge by Date Hour
#df_f.tail(30)

In [44]:
df_f.shape # combined dataframe of energy + meteomatics city weather variables 138791 rows

(138791, 89)

In [45]:
#create list for city name columns
cit_list = [col for col in df_f.columns if col.startswith('City')]
print(cit_list) #verify

['City_Cul', 'City_Ca', 'City_Herm', 'City_Guas', 'City_Guay', 'City_Maz', 'City_Moch', 'City_Nav', 'City_Nog', 'City_Obr']


In [46]:
#drop city name columns
df_f.drop(columns=(cit_list), axis = 1,  inplace=True)
df_f.head(2)

,Date,Energy_demand,Date_time,Hour,Week_day,Week,Month,Year,Season,Temperature_Cul,...,Wind_Velocity_Nog,Solar_Radiation_Nog,Cloudiness_Nog,Temperature_Obr,Perceived_Temperature_Obr,Precipitation_Obr,Humidity_Obr,Wind_Velocity_Obr,Solar_Radiation_Obr,Cloudiness_Obr
0,2007-01-01,1297.0,2007-01-01 01:00:00,1,0,1,1,2007,Winter,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-01-01,1255.0,2007-01-01 02:00:00,2,0,1,1,2007,Winter,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
os.chdir("..") # go back to up folder

In [51]:
print(os.getcwd())


/Users/yhmve/nrgy_feng/2023_wip


In [52]:
#send combined dataframe to file
df_f.to_csv (r'df_all.csv', index = False)